### Model Training EDA in Jupyter notebook

##### 1.1 importing data and requirements 

##### pandas, Matplotlib, numpy, seaborn, warning module, sklearn

In [29]:
# basic imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# modelling iports
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer


#### importing data

In [30]:
data_frame = pd.read_csv("/Users/dhiremath/02 pesonal/Personal docs/ML_project_youtube_Krish_naik/notebook/data/stud.csv")
data_frame.head(5 )

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


#### preparing X and Y variables 

In [31]:
X = data_frame.drop(columns =["math_score"], axis=1)
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [32]:
Y = data_frame["math_score"]
Y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

In [33]:
print("different categories in non numerical colums" , end= ":")

print("gender: ", end ="")
print(data_frame['gender'].unique())
print("race_ethnicity: ", end ="")
print(data_frame['race_ethnicity'].unique())
print("parental_level_of_education: ", end ="")
print(data_frame['parental_level_of_education'].unique())
print("lunch: ", end ="")
print(data_frame['lunch'].unique())
print("test_preparation_course: ", end ="")
print(data_frame['test_preparation_course'].unique())





different categories in non numerical colums:gender: ['female' 'male']
race_ethnicity: ['group B' 'group C' 'group A' 'group D' 'group E']
parental_level_of_education: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
lunch: ['standard' 'free/reduced']
test_preparation_course: ['none' 'completed']


In [34]:
# preparing the data into categorical and numerical data for  one hot coding and standardizaion

num_features = X.select_dtypes(exclude ='object').columns

cat_features = X.select_dtypes(include=["object"]).columns
# num_features
# cat_features

In [35]:
num_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("numerical columns transformation", num_transformer, num_features),
        ("categorical columns transform", cat_transformer, cat_features)
    ]
)
preprocessor

ColumnTransformer(transformers=[('numerical columns transformation',
                                 StandardScaler(),
                                 Index(['reading_score', 'writing_score'], dtype='object')),
                                ('categorical columns transform',
                                 OneHotEncoder(),
                                 Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course'],
      dtype='object'))])

In [36]:
X = preprocessor.fit_transform(X)
X.shape

/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Ch

(1000, 19)

In [37]:
# splittign the data for test and train

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size= .2, random_state= 42) 
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((800, 19), (200, 19), (800,), (200,))

#### creating evaluation function to run after model training

In [38]:
def model_evaliation(true, predicted):
    mae=   mean_absolute_error(true, predicted)
    #mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_squared = r2_score(true, predicted)

    return mae,rmse,r2_squared


##### Setting up the models list 

In [40]:
# list of models to apply in loop and compare its perfoormance

models = {
    "Linear Regression": LinearRegression(),
    "lasso": Lasso(),
    "ridge": Ridge(),
    "decision Tree": DecisionTreeRegressor(),
    "Randon Forest": RandomForestRegressor(),
    "K-Neighbors regression": KNeighborsRegressor(),
    "Support vector machine" : SVR(),
    "ADA Boost": AdaBoostRegressor(),
    "XG boost Regression": XGBRegressor(),
    "cat Boost Regression": CatBoostRegressor()
}

# to collect the models and the resulting scores
models_list =[]
mae_results_list =[]
r2_squared_results_list= []
rmse_result_list =[]

# applying the models in loop

for i in range(len(list(models))):
    model = list (models.values())[i]
    model.fit(X_train, Y_train) # trainign the model

    # predicting the output
    Y_train_predict = model.predict(X_train) 
    Y_test_predict = model.predict(X_test) 

    #evaluate model performance
    mae_train,rmse_train,r2_squared_train = model_evaliation(Y_train,Y_train_predict)
    mae,rmse,r2_squared = model_evaliation(Y_test,Y_test_predict)

    model_name = list(models.keys())[i]
    models_list.append(model_name)
    r2_squared_results_list.append(r2_squared)
    rmse_result_list.append(rmse)
    mae_results_list.append(mae)

    print("performance indicators for model:{0} for training set are: mae = {1}, rmse = {2}, r2_squared = {3}".format(model_name,mae_train, rmse_train,r2_squared_train))
    print("performance indicators for model:{0} for test set are: mae = {1}, rmse = {2}, r2_squared = {3}".format(model_name,mae, rmse,r2_squared))

/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.


performance indicators for model:Linear Regression for training set are: mae = 4.266711846071956, rmse = 5.323050852720514, r2_squared = 0.8743172040139593
performance indicators for model:Linear Regression for test set are: mae = 4.21476314247485, rmse = 5.393993869732841, r2_squared = 0.8804332983749565
performance indicators for model:lasso for training set are: mae = 5.206296077972952, rmse = 6.593807540619166, r2_squared = 0.8071466723085148
performance indicators for model:lasso for test set are: mae = 5.157879138921816, rmse = 6.519688056285626, r2_squared = 0.825320079562973
performance indicators for model:ridge for training set are: mae = 4.26498782372598, rmse = 5.323324922741654, r2_squared = 0.8743042615212909
performance indicators for model:ridge for test set are: mae = 4.211100688014258, rmse = 5.390387016935635, r2_squared = 0.880593148502874
performance indicators for model:decision Tree for training set are: mae = 0.01875, rmse = 0.2795084971874737, r2_squared = 0.99

/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.


performance indicators for model:Randon Forest for training set are: mae = 1.8330333333333335, rmse = 2.3066690811644395, r2_squared = 0.9763992832837575
performance indicators for model:Randon Forest for test set are: mae = 4.687283333333333, rmse = 6.036614676290014, r2_squared = 0.8502467479444263
performance indicators for model:K-Neighbors regression for training set are: mae = 4.51675, rmse = 5.712753276660913, r2_squared = 0.8552410116072555
performance indicators for model:K-Neighbors regression for test set are: mae = 5.619, rmse = 7.253757646902741, r2_squared = 0.7837702557426202


/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.


performance indicators for model:Support vector machine for training set are: mae = 4.869189452384868, rmse = 6.57700724251018, r2_squared = 0.8081281585902299
performance indicators for model:Support vector machine for test set are: mae = 5.401539244496997, rmse = 8.126623218622633, r2_squared = 0.7286001513223705
performance indicators for model:ADA Boost for training set are: mae = 4.83460196112691, rmse = 5.875843394491787, r2_squared = 0.8468577484773365
performance indicators for model:ADA Boost for test set are: mae = 4.7812350376513235, rmse = 6.088924675433132, r2_squared = 0.8476401435075661


/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.


performance indicators for model:XG boost Regression for training set are: mae = 0.6147581052780151, rmse = 0.9087352405171366, r2_squared = 0.996337063889075
performance indicators for model:XG boost Regression for test set are: mae = 5.070335817337036, rmse = 6.621583071401701, r2_squared = 0.8198173310440375
Learning rate set to 0.039525
0:	learn: 14.5987177	total: 59.5ms	remaining: 59.5s
1:	learn: 14.2251886	total: 60.2ms	remaining: 30s
2:	learn: 13.8866124	total: 60.8ms	remaining: 20.2s
3:	learn: 13.5235688	total: 61.4ms	remaining: 15.3s
4:	learn: 13.1887021	total: 61.9ms	remaining: 12.3s
5:	learn: 12.9124226	total: 62.4ms	remaining: 10.3s
6:	learn: 12.6000335	total: 62.8ms	remaining: 8.91s
7:	learn: 12.3299057	total: 63.2ms	remaining: 7.83s
8:	learn: 12.0660619	total: 63.6ms	remaining: 7s
9:	learn: 11.7730981	total: 64.1ms	remaining: 6.35s
10:	learn: 11.4922764	total: 64.7ms	remaining: 5.82s
11:	learn: 11.2626483	total: 65.1ms	remaining: 5.36s
12:	learn: 11.0426039	total: 65.5ms	

/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dhiremath/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
